## Keras Tuner - Deciding Number of Hidden Layers and Neuron in Neural Network

In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Shape of Dataset
df.shape

(10000, 14)

In [5]:
# Independnet Feature
X = df.iloc[:, 3:13]

# Dependent Feature
y = df.iloc[:, 13]

print("X Shape:"+str(X.shape))
print("y Shape:"+str(y.shape))

X Shape:(10000, 10)
y Shape:(10000,)


In [6]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [7]:
## Concatenate the Data Frames
X=pd.concat([X,geography,gender],axis=1)
        
## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [8]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [9]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### # Perform Hyperparameter Optimization

In [10]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [11]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    


In [18]:
model = KerasClassifier(build_fn=create_model, verbose=True)

C:\Users\Jeetr\AppData\Local\Temp/ipykernel_7728/1859348228.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=True)


In [19]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [20]:
grid_result = grid.fit(X_train, y_train)

Epoch 1/30
50/50 [==============================] - 0s 2ms/step - loss: 1.0595 - accuracy: 0.2409
Epoch 2/30
50/50 [==============================] - 0s 2ms/step - loss: 0.8134 - accuracy: 0.4008
Epoch 3/30
50/50 [==============================] - 0s 2ms/step - loss: 0.6476 - accuracy: 0.6186
Epoch 4/30
50/50 [==============================] - 0s 2ms/step - loss: 0.5587 - accuracy: 0.7345
Epoch 5/30
50/50 [==============================] - 0s 2ms/step - loss: 0.5203 - accuracy: 0.7672
Epoch 6/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4892 - accuracy: 0.7948
Epoch 7/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4741 - accuracy: 0.8012
Epoch 8/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4704 - accuracy: 0.7994
Epoch 9/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4591 - accuracy: 0.8037
Epoch 10/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4549 - accuracy: 0.8036
Epoch 11/30
50/50 [

50/50 [==============================] - 0s 2ms/step - loss: 0.4474 - accuracy: 0.8039
Epoch 23/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4497 - accuracy: 0.8005
Epoch 24/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4483 - accuracy: 0.8028
Epoch 25/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4442 - accuracy: 0.8061
Epoch 26/30
50/50 [==============================] - 0s 1ms/step - loss: 0.4441 - accuracy: 0.8030
Epoch 27/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4439 - accuracy: 0.8006
Epoch 28/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4451 - accuracy: 0.8020
Epoch 29/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4441 - accuracy: 0.8005
Epoch 30/30
13/13 [==============================] - 0s 2ms/step - loss: 0.4011 - accuracy: 0.8263
Epoch 1/30
50/50 [==============================] - 0s 2ms/step - loss: 0.9602 - accuracy: 0.2975
Epoch 2/30
50/50 [=====

50/50 [==============================] - 0s 2ms/step - loss: 0.4532 - accuracy: 0.8019
Epoch 13/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4468 - accuracy: 0.8023
Epoch 14/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4525 - accuracy: 0.8037
Epoch 15/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4445 - accuracy: 0.8055
Epoch 16/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4439 - accuracy: 0.8027
Epoch 17/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4441 - accuracy: 0.8025
Epoch 18/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4429 - accuracy: 0.8045
Epoch 19/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4447 - accuracy: 0.8072
Epoch 20/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4393 - accuracy: 0.8058
Epoch 21/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4408 - accuracy: 0.8041
Epoch 22/30
50/50 [===

50/50 [==============================] - 0s 2ms/step - loss: 0.5297 - accuracy: 0.7887
Epoch 3/30
50/50 [==============================] - 0s 2ms/step - loss: 0.5125 - accuracy: 0.7952
Epoch 4/30
50/50 [==============================] - 0s 2ms/step - loss: 0.5077 - accuracy: 0.7967
Epoch 5/30
50/50 [==============================] - 0s 2ms/step - loss: 0.5043 - accuracy: 0.7956
Epoch 6/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4938 - accuracy: 0.7964
Epoch 7/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4829 - accuracy: 0.7966
Epoch 8/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4778 - accuracy: 0.7986
Epoch 9/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4708 - accuracy: 0.7991
Epoch 10/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4664 - accuracy: 0.7992
Epoch 11/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4604 - accuracy: 0.8030
Epoch 12/30
50/50 [==========

50/50 [==============================] - 0s 3ms/step - loss: 0.4389 - accuracy: 0.8027
Epoch 24/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4433 - accuracy: 0.8020
Epoch 25/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4375 - accuracy: 0.7980
Epoch 26/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4414 - accuracy: 0.8022
Epoch 27/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4376 - accuracy: 0.8083
Epoch 28/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4399 - accuracy: 0.8067
Epoch 29/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4413 - accuracy: 0.7995
Epoch 30/30
13/13 [==============================] - 0s 2ms/step - loss: 0.4289 - accuracy: 0.8069
Epoch 1/30
50/50 [==============================] - 1s 3ms/step - loss: 0.5379 - accuracy: 0.7800
Epoch 2/30
50/50 [==============================] - 0s 2ms/step - loss: 0.5229 - accuracy: 0.7952
Epoch 3/30
50/50 [======

...

25/25 [==============================] - 0s 2ms/step - loss: 0.5293 - accuracy: 0.7847
Epoch 8/30
25/25 [==============================] - 0s 2ms/step - loss: 0.5133 - accuracy: 0.7909
Epoch 9/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4945 - accuracy: 0.7927
Epoch 10/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4910 - accuracy: 0.7945
Epoch 11/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4823 - accuracy: 0.7947
Epoch 12/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4700 - accuracy: 0.7989
Epoch 13/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4657 - accuracy: 0.7987
Epoch 14/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4595 - accuracy: 0.8019
Epoch 15/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4572 - accuracy: 0.8036
Epoch 16/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4492 - accuracy: 0.8053
Epoch 17/30
25/25 [=====

25/25 [==============================] - 0s 3ms/step - loss: 0.3949 - accuracy: 0.8350
Epoch 29/30
25/25 [==============================] - 0s 3ms/step - loss: 0.3923 - accuracy: 0.8389
Epoch 30/30
7/7 [==============================] - 0s 2ms/step - loss: 0.3715 - accuracy: 0.8481
Epoch 1/30
25/25 [==============================] - 1s 3ms/step - loss: 0.5977 - accuracy: 0.7038
Epoch 2/30
25/25 [==============================] - 0s 3ms/step - loss: 0.5234 - accuracy: 0.7847
Epoch 3/30
25/25 [==============================] - 0s 3ms/step - loss: 0.5001 - accuracy: 0.7914
Epoch 4/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4833 - accuracy: 0.7934
Epoch 5/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4687 - accuracy: 0.7980
Epoch 6/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4634 - accuracy: 0.8000
Epoch 7/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4562 - accuracy: 0.8025
Epoch 8/30
25/25 [=============

25/25 [==============================] - 0s 3ms/step - loss: 0.4175 - accuracy: 0.8248
Epoch 19/30
25/25 [==============================] - 0s 4ms/step - loss: 0.4135 - accuracy: 0.8236
Epoch 20/30
25/25 [==============================] - 0s 4ms/step - loss: 0.4083 - accuracy: 0.8320
Epoch 21/30
25/25 [==============================] - 0s 4ms/step - loss: 0.4060 - accuracy: 0.8333
Epoch 22/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4112 - accuracy: 0.8294
Epoch 23/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4065 - accuracy: 0.8292
Epoch 24/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4056 - accuracy: 0.8344
Epoch 25/30
25/25 [==============================] - 0s 3ms/step - loss: 0.3962 - accuracy: 0.8397
Epoch 26/30
25/25 [==============================] - 0s 3ms/step - loss: 0.3970 - accuracy: 0.8363
Epoch 27/30
25/25 [==============================] - 0s 4ms/step - loss: 0.3925 - accuracy: 0.8369
Epoch 28/30
25/25 [===

25/25 [==============================] - 0s 4ms/step - loss: 0.4523 - accuracy: 0.8009
Epoch 9/30
25/25 [==============================] - 0s 4ms/step - loss: 0.4473 - accuracy: 0.8012
Epoch 10/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4520 - accuracy: 0.8047
Epoch 11/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4402 - accuracy: 0.8053
Epoch 12/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4378 - accuracy: 0.8105
Epoch 13/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4362 - accuracy: 0.8106
Epoch 14/30
25/25 [==============================] - 0s 4ms/step - loss: 0.4259 - accuracy: 0.8145
Epoch 15/30
25/25 [==============================] - 0s 4ms/step - loss: 0.4273 - accuracy: 0.8144
Epoch 16/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4253 - accuracy: 0.8164
Epoch 17/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4195 - accuracy: 0.8231
Epoch 18/30
25/25 [====

25/25 [==============================] - 0s 3ms/step - loss: 0.3928 - accuracy: 0.8395
Epoch 30/30
7/7 [==============================] - 1s 4ms/step - loss: 0.3732 - accuracy: 0.8462
Epoch 1/30
25/25 [==============================] - 1s 4ms/step - loss: 0.5789 - accuracy: 0.7430
Epoch 2/30
25/25 [==============================] - 0s 3ms/step - loss: 0.5100 - accuracy: 0.7933
Epoch 3/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4900 - accuracy: 0.7953
Epoch 4/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4787 - accuracy: 0.7975
Epoch 5/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4633 - accuracy: 0.7994
Epoch 6/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4665 - accuracy: 0.8002
Epoch 7/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4478 - accuracy: 0.8034
Epoch 8/30
25/25 [==============================] - 0s 3ms/step - loss: 0.4456 - accuracy: 0.8061
Epoch 9/30
25/25 [==============

In [21]:
[grid_result.best_score_,grid_result.best_params_]

[0.8571250081062317,
 {'activation': 'relu',
  'batch_size': 128,
  'epochs': 30,
  'layers': [45, 30, 15]}]

In [22]:
y_pred=grid.predict(X_test)

63/63 [==============================] - 0s 2ms/step


In [23]:
from sklearn.metrics import confusion_matrix,accuracy_score

cm = confusion_matrix(y_test, y_pred)
cm

array([[1529,   66],
       [ 208,  197]], dtype=int64)

In [24]:
score=accuracy_score(y_test,y_pred)
score

0.863